In [1]:
!pip install pybullet==2.5.5

%cd /usr/local/lib/python3.6/dist-packages
!git clone https://github.com/benelot/pybullet-gym.git
%cd pybullet-gym
!pip install -e .

/usr/local/lib/python3.6/dist-packages
fatal: destination path 'pybullet-gym' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pybullet-gym
Obtaining file:///usr/local/lib/python3.6/dist-packages/pybullet-gym
  Found existing installation: pybulletgym 0.1
    Can't uninstall 'pybulletgym'. No files were found to uninstall.
  Running setup.py develop for pybulletgym


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import gym
import pybulletgym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
from tqdm import trange
import pickle

import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [4]:
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()

In [5]:
%cd /content
%mkdir drive/My\ Drive/project_evo/comparation
%mkdir drive/My\ Drive/project_evo/comparation/video

/content
mkdir: cannot create directory ‘drive/My Drive/project_evo/comparation’: File exists
mkdir: cannot create directory ‘drive/My Drive/project_evo/comparation/video’: File exists


In [6]:
def show_video(folder):
    mp4list = glob.glob(f'drive/My Drive/project_evo/comparation/video/{folder}/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 300px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env, folder):
    env = Monitor(env, f'drive/My Drive/project_evo/comparation/video/{folder}', force=True)
    return env

In [7]:
CUDA = torch.cuda.is_available()

def to_np(x):
    return x.detach().cpu().numpy()

def to_tensor(x, requires_grad=False):
    x = torch.from_numpy(x)
    if CUDA:
        x = x.cuda()
    
    if requires_grad:
        return x.clone().contiguous().detach().requires_grad_(True)
    else:
        return x.clone().contiguous().detach()

    
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(111, 100),
            nn.Tanh(),
            nn.Linear(100, 8),
            nn.Tanh()
        )

        
    def forward(self, X):
        X = X.view(X.size(0), -1)
        return self.layers.forward(X)
    
 
    def set_params(self, params):
        cpt = 0
        for param in self.parameters():
            tmp = np.product(param.size())

            if torch.cuda.is_available():
                param.data.copy_(to_tensor(
                    params[cpt:cpt + tmp]).view(param.size()).cuda())
            else:
                param.data.copy_(to_tensor(
                    params[cpt:cpt + tmp]).view(param.size()))
            cpt += tmp

In [8]:
def test(weights, filename):
    actor.set_params(weights)
    env = wrap_env(gym.make("AntMuJoCoEnv-v0"), filename)
    observation = env.reset()
    observation = to_tensor(observation.reshape(1, -1).astype(np.float32))

    iters = 2500
    env.env._max_episode_steps = iters

    for _ in trange(iters, position=0, leave=True):
        env.render()
        action = actor(observation)
        action = to_np(action)[0]
        
        observation, reward, done, info = env.step(action)
        observation = to_tensor(observation.reshape(1, -1).astype(np.float32))
        if done: 
            break;
                
    env.close()

In [9]:
from_gen = pickle.load(open('drive/My Drive/project_evo/history/from_gen/best.pkl', 'rb'))
from_data = pickle.load(open('drive/My Drive/project_evo/history/from_data/best.pkl', 'rb'))
first = pickle.load(open('drive/My Drive/project_evo/history/1300.pkl', 'rb'))['best']
second = pickle.load(open('drive/My Drive/project_evo/history/3000.pkl', 'rb'))['best']

In [10]:
actor = Network()

In [11]:
test(from_gen, 'from_gen')
test(from_data, 'from_data')
test(first, 'first')
test(second, 'second')

  0%|          | 0/2500 [00:00<?, ?it/s]

current_dir=/usr/local/lib/python3.6/dist-packages/pybullet_envs/bullet
WalkerBase::__init__
options= 


  0%|          | 0/2500 [00:00<?, ?it/s]

WalkerBase::__init__
options= 


  0%|          | 0/2500 [00:00<?, ?it/s]

WalkerBase::__init__
options= 


  0%|          | 0/2500 [00:00<?, ?it/s]

WalkerBase::__init__
options= 


100%|█████████▉| 2499/2500 [01:31<00:00, 27.35it/s]

In [12]:
show_video('from_gen')

In [13]:
show_video('from_data')

In [14]:
show_video('first')

In [15]:
show_video('second')